In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [12]:
polls = pd.read_csv('president_general_polls_2016.csv')

polls.head(5)

,cycle,branch,type,matchup,forecastdate,state,startdate,enddate,pollster,grade,...,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,multiversions,url,poll_id,question_id,createddate,timestamp
0,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/3/2016,11/6/2016,ABC News/Washington Post,A+,...,45.20163,41.72430,4.626221,NaN,NaN,https://www.washingtonpost.com/news/the-fix/wp...,48630,76192,11/7/16,09:35:33 8 Nov 2016
1,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/1/2016,11/7/2016,Google Consumer Surveys,B,...,43.34557,41.21439,5.175792,NaN,NaN,https://datastudio.google.com/u/0/#/org//repor...,48847,76443,11/7/16,09:35:33 8 Nov 2016
2,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/2/2016,11/6/2016,Ipsos,A-,...,42.02638,38.81620,6.844734,NaN,NaN,http://projects.fivethirtyeight.com/polls/2016...,48922,76636,11/8/16,09:35:33 8 Nov 2016
3,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/4/2016,11/7/2016,YouGov,B,...,45.65676,40.92004,6.069454,NaN,NaN,https://d25d2506sfb94s.cloudfront.net/cumulus_...,48687,76262,11/7/16,09:35:33 8 Nov 2016
4,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/8/16,U.S.,11/3/2016,11/6/2016,Gravis Marketing,B-,...,46.84089,42.33184,3.726098,NaN,NaN,http://www.gravispolls.com/2016/11/final-natio...,48848,76444,11/7/16,09:35:33 8 Nov 2016


In [13]:
""" Clean 538 polls
-- cleaning 2016 polling
"""

groups = polls.groupby(['state'])
state_dfs = {}
for state, group in groups:
    state_dfs[state] = group

state_keys = list(state_dfs.keys())

for i in state_keys:
    state_dfs[i]['startdate'] = pd.to_datetime(state_dfs[i]['startdate'])
    state_dfs[i]['enddate'] = pd.to_datetime(state_dfs[i]['enddate'])

    state_dfs[i]['middate'] = state_dfs[i]['startdate'] + (state_dfs[i]['enddate'] - state_dfs[i]['startdate'])/2

ref_date = datetime(2016, 11, 8)
one_month_before = ref_date - timedelta(days=30)

for j in state_keys:
    state_dfs[j].sort_values(by='middate', inplace=True)
    state_dfs[j] = state_dfs[j][state_dfs[j]['middate'].between(one_month_before, ref_date)]
    state_dfs[j].reset_index(drop=True, inplace=True)

poll_lst = []

for key, df in state_dfs.items():

    state_name = key
    average_poll_value = df['adjpoll_clinton'].mean()
    poll_lst.append([state_name, average_poll_value])

poll_avgs_2016 = pd.DataFrame(poll_lst, columns=['state', 'average'])

poll_avgs_2016['state'] = poll_avgs_2016['state'].astype(str)
def extract_state_name(value):
    return eval(value)[0]

poll_avgs_2016['state'] = poll_avgs_2016['state'].apply(extract_state_name)

bad_rows = ['District of Columbia', 'Maine CD-1', 'Maine CD-2', 'Nebraska CD-1', 'Nebraska CD-2', 'Nebraska CD-3', 'U.S.']
poll_avgs_2016 = poll_avgs_2016[~poll_avgs_2016['state'].isin(bad_rows)].reset_index(drop=True)

In [14]:
poll_avgs_2016.head(10)

,state,average
0,Alabama,33.453993
1,Alaska,35.387560
2,Arizona,42.331856
3,Arkansas,34.462234
4,California,55.694156
5,Colorado,44.018925
6,Connecticut,49.034463
7,Delaware,47.700368
8,Florida,45.774647
9,Georgia,42.465996
